# Secondary Mushroom Dataset Analysis

In [1]:
# import of all the necessary libraries
import urllib
import zipfile
import os

In [2]:
# Defining the function to download and extract the file, returns the path to the extracted file

def download_and_extract_file(url, destination) -> str:
    """
    Download and extract url to destination

    :param url: url to download
    :param destination: destination path

    :return: path to extracted file
    """
    urllib.request.urlretrieve(url, destination)
    with zipfile.ZipFile(destination, "r") as zip_ref:
        zip_ref.extractall("./")
    os.remove(destination)
    name = "MushroomDataset.zip"
    with zipfile.ZipFile(name, "r") as zip_ref:
        zip_ref.extractall("./")
    os.remove(name)
    res = os.listdir("./")
    for i in res:
        if i == "MushroomDataset":
            return "./MushroomDataset/"
    return "./"

In [3]:
source = "https://archive.ics.uci.edu/static/public/848/secondary+mushroom+dataset.zip"
destination_zip = "./secondary+mushroom+dataset.zip"

datasets_path = download_and_extract_file(source, destination_zip)
print("Dataset downloaded and extracted to: " + datasets_path + "\n")

Dataset downloaded and extracted to: ./MushroomDataset/



### Transforming data from csv to DataFrame

In [4]:
df = spark.read.format("csv").load(datasets_path + "secondary_data.csv", header=True, inferSchema=True, sep=";")
df.show(5)

+-----+------------+---------+-----------+---------+--------------------+---------------+------------+----------+-----------+----------+---------+------------+----------+---------+----------+--------+---------+-----------------+-------+------+
|class|cap-diameter|cap-shape|cap-surface|cap-color|does-bruise-or-bleed|gill-attachment|gill-spacing|gill-color|stem-height|stem-width|stem-root|stem-surface|stem-color|veil-type|veil-color|has-ring|ring-type|spore-print-color|habitat|season|
+-----+------------+---------+-----------+---------+--------------------+---------------+------------+----------+-----------+----------+---------+------------+----------+---------+----------+--------+---------+-----------------+-------+------+
|    p|       15.26|        x|          g|        o|                   f|              e|        NULL|         w|      16.95|     17.09|        s|           y|         w|        u|         w|       t|        g|             NULL|      d|     w|
|    p|        16.6|    

#### Visualization of the dataset

In [5]:
# We produce some statistics about the dataset
print("Number of rows: " + str(df.count()))
print("Number of columns: " + str(len(df.columns)))
print("Columns: " + str(df.columns))
print("Schema: ")
df.printSchema()

Number of rows: 61069
Number of columns: 21
Columns: ['class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color', 'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color', 'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color', 'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color', 'habitat', 'season']
Schema: 
root
 |-- class: string (nullable = true)
 |-- cap-diameter: double (nullable = true)
 |-- cap-shape: string (nullable = true)
 |-- cap-surface: string (nullable = true)
 |-- cap-color: string (nullable = true)
 |-- does-bruise-or-bleed: string (nullable = true)
 |-- gill-attachment: string (nullable = true)
 |-- gill-spacing: string (nullable = true)
 |-- gill-color: string (nullable = true)
 |-- stem-height: double (nullable = true)
 |-- stem-width: double (nullable = true)
 |-- stem-root: string (nullable = true)
 |-- stem-surface: string (nullable = true)
 |-- stem-color: string (nullable = true)
 |-- veil-type: string (

In [7]:
# We check if there are any null values in the dataset
from pyspark.sql.functions import count, when, isnan, col
print("Number of null values in each column: ")
df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()


Number of null values in each column: 


+-----+------------+---------+-----------+---------+--------------------+---------------+------------+----------+-----------+----------+---------+------------+----------+---------+----------+--------+---------+-----------------+-------+------+
|class|cap-diameter|cap-shape|cap-surface|cap-color|does-bruise-or-bleed|gill-attachment|gill-spacing|gill-color|stem-height|stem-width|stem-root|stem-surface|stem-color|veil-type|veil-color|has-ring|ring-type|spore-print-color|habitat|season|
+-----+------------+---------+-----------+---------+--------------------+---------------+------------+----------+-----------+----------+---------+------------+----------+---------+----------+--------+---------+-----------------+-------+------+
|    0|           0|        0|      14120|        0|                   0|           9884|       25063|         0|          0|         0|    51538|       38124|         0|    57892|     53656|       0|     2471|            54715|      0|     0|
+-----+------------+----

In [10]:
# We remove duplicates from the dataset
print("Number of rows before removing duplicates: " + str(df.count()))
df = df.dropDuplicates()
print("Number of rows after removing duplicates: " + str(df.count()))

Number of rows before removing duplicates: 61069
Number of rows after removing duplicates: 60923


In [11]:
# We compute the statistics of the dataset
print("Statistics of the dataset: ")
df.describe().show()

Statistics of the dataset: 


23/12/19 11:14:05 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+-------+-----+------------------+---------+-----------+---------+--------------------+---------------+------------+----------+-----------------+------------------+---------+------------+----------+---------+----------+--------+---------+-----------------+-------+------+
|summary|class|      cap-diameter|cap-shape|cap-surface|cap-color|does-bruise-or-bleed|gill-attachment|gill-spacing|gill-color|      stem-height|        stem-width|stem-root|stem-surface|stem-color|veil-type|veil-color|has-ring|ring-type|spore-print-color|habitat|season|
+-------+-----+------------------+---------+-----------+---------+--------------------+---------------+------------+----------+-----------------+------------------+---------+------------+----------+---------+----------+--------+---------+-----------------+-------+------+
|  count|60923|             60923|    60923|      46803|    60923|               60923|          51068|       35861|     60923|            60923|             60923|     9387|       228

In [19]:
# We delete the columns with a high number of null values, greater than 50% of the total number of rows
print("Number of rows before removing columns with a high number of null values: " + str(df.count()))

# We compute the number of null values in each column
null_values = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0]
null_values_dict = {}
for i in range(len(null_values)):
    if null_values[i] == 0:
        continue
    null_values_dict[null_values[i]] = df.columns[i]


Number of rows before removing columns with a high number of null values: 60923


{14120: 'cap-surface',
 9855: 'gill-attachment',
 25062: 'gill-spacing',
 51536: 'stem-root',
 38122: 'stem-surface',
 57746: 'veil-type',
 53510: 'veil-color',
 2471: 'ring-type',
 54597: 'spore-print-color'}

In [21]:
# From the data above we decide to remove the columns with a high number of null values, greater than 50% of the total number of rows
null_values_to_remove = []
for i in null_values_dict:
    if i > df.count() / 2:
        null_values_to_remove.append(null_values_dict[i])

df = df.drop(*null_values_to_remove)
print("Number of rows after removing columns with a high number of null values: " + str(df.count()))
df.columns

Number of rows after removing columns with a high number of null values: 60923


['class',
 'cap-diameter',
 'cap-shape',
 'cap-surface',
 'cap-color',
 'does-bruise-or-bleed',
 'gill-attachment',
 'gill-spacing',
 'gill-color',
 'stem-height',
 'stem-width',
 'stem-color',
 'has-ring',
 'ring-type',
 'habitat',
 'season']

In [25]:
# For all the other columns with a low number of null values, we replace the null values with the most frequent value of the column
print("Number of rows before replacing null values: " + str(df.count()))
for i in df.columns:
    if df.filter(df[i].isNull()).count() > 0:
        most_frequent_value = df.groupBy(i).count().orderBy("count", ascending=False).collect()
        if most_frequent_value[0][0] == None:
            most_frequent_value = most_frequent_value[1][0]
        else:
            most_frequent_value = most_frequent_value[0][0]
        print("Most frequent value of column " + i + ": " + str(most_frequent_value))
        df = df.fillna(most_frequent_value, subset=[i])


Number of rows before replacing null values: 60923
Most frequent value of column cap-surface: t
Most frequent value of column gill-attachment: a
Most frequent value of column gill-spacing: c
Most frequent value of column ring-type: f


In [26]:
# We compute the number of null values in each column
null_values = df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).collect()[0]
null_values_dict = {}
for i in range(len(null_values)):
    if null_values[i] == 0:
        continue
    null_values_dict[null_values[i]] = df.columns[i]

{}